## 📊 分析の流れ
<details>
<summary><b>目次</b></summary>

1. **データ読み込み**
1. **欠損値処理**
    - 欠損列処理  
    - 欠損値処理  
    - 異常値処理  
1. **集計**
1. **データ出力**
</details>

In [ ]:
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import src.config as cfg
import src.data_loader as dl

## 📥 設定値の読み込み
yamlファイルから設定を読み込みます。また、プレースホルダやゼロをチェックします。


In [ ]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
#display(settings)

## 🔍 欠損列処理
4つファイルの列が各年度で同じかを目視で確認しておきます。
問題がある場合は処理します。
<details><summary><b>結果</b></summary>
すべてのファイルで同じ列が登録されています。問題ありません。

- fy-balance-sheet.csv
- fy-cash-flow-statement.csv
- fy-profit-and-loss.csv
- fy-stock-dividend.csv
</details>


In [ ]:
dl.load_yearly_header(settings["data_path"], settings["years"], settings["files"])

## 📥 欠損値表現のチェック
プレースホルダやゼロをチェックし、NaNのデータを特定します。  
NaNのデータは、setting.yamlの`na_values`で指定します。

```python


In [ ]:
# nan_valuesに""だけ指定し、他のプレースホルダをチェックする"
df = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"], na_values=[""])
#display(all_df)
counts = []
for filename in df:
    for col in df[filename]:
        #print(col)
        ser = df[filename][col].fillna('')
        counts.append(
            (filename, col,
            (ser == '-').sum(),
            (ser == '0').sum(),
            (ser == '—').sum(),
            ser.str.contains('[A-Za-z]', na=False).sum())
            )
placeholder_counts = pd.DataFrame(counts, columns=["filename","col","ハイフン半角","ゼロ","ハイフン全角","alphabet"])
#display(placeholder_counts)


## 📥 データ読み込み

In [ ]:
all_df = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"], settings["na_values"])
#display(all_df[settings["files"][0]].head(1000))

,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
0,1301,2010/03,6.430100e+10,1.853800e+10,1.839100e+10,1.258900e+10,2.672400e+10,2.712000e+09,1651.87,28.1
1,1332,2010/03,3.839240e+11,7.956700e+10,6.976600e+10,3.253000e+10,1.146390e+11,1.069930e+11,231.06,16.0
2,1333,2010/03,1.136840e+11,2.443500e+10,NaN,NaN,NaN,NaN,NaN,21.5
3,1352,2010/03,1.196700e+10,5.439000e+09,5.268000e+09,1.561000e+09,NaN,4.970000e+08,628.95,44.0
4,1375,2010/03,3.208300e+10,6.318000e+09,6.307000e+09,3.056000e+09,NaN,7.786000e+09,1.62,19.0
...,...,...,...,...,...,...,...,...,...,...
995,3952,2010/03,9.146018e+09,6.999483e+09,6.991998e+09,5.001907e+09,NaN,NaN,1339.61,76.5
996,3953,2010/03,5.970935e+09,4.057037e+09,4.052110e+09,2.972933e+09,NaN,NaN,830.68,67.9
997,3954,2010/03,1.912368e+10,8.663503e+09,7.800986e+09,6.875919e+09,2.826280e+09,8.080000e+08,1888.95,44.0
998,3955,2010/01,2.168600e+10,1.166000e+10,1.160700e+10,9.071000e+09,1.980000e+09,1.304000e+09,1084.50,53.7


## 🧹 欠損値処理
ここでは、後の分析・スクリーニングの処理を適切に設計するため、欠損値の傾向を確認します。  
⚠️ **注意※** csvファイルには欠損値に"-"が入っているため、NaNに置き換えて読み込みます。
<details><summary><b>分析内容</b></summary>

- 列ごとの欠損値の数・割合を確認する
- 年ごとの欠損率の傾向を確認
- 全企業の平均欠損率の分布から特徴を詳細分析
<br>
</details>
<details><summary><b>結果と考察</b></summary>

- fy-balance-sheet.csv
- fy-cash-flow-statement.csv
- fy-profit-and-loss.csv
- fy-stock-dividend.csv
</details>


### 列ごとの欠損値の割合を確認する

In [ ]:

missing_column_df = {}
for filename in files:
    if filename not in missing_column_df:
        missing_column_df[filename] = pd.DataFrame()
    missing_column_df[filename] = all_df[filename].isna().mean()
    fig = px.bar(missing_column_df[filename], x=missing_column_df[filename].index,
                    y=missing_column_df[filename].values)
    fig.show()

### 年ごとの欠損率の傾向を確認（折れ線グラフ）

In [ ]:
#missing_year_df = {}
pd.options.display.max_rows = 1000
#display(all_df[settings["files"][0]].head(1000))
df = all_df[settings["files"][0]].copy()
df["年度_年"] = df["年度"].astype(str).str[:4].astype(int)

missing_ratio_by_year = (
    df.drop(columns=["コード", "年度"])
      .groupby("年度_年")
      .agg(lambda g: g.isnull().mean())
      .reset_index()
)
display(missing_ratio_by_year)
fig = px.line(
    missing_ratio_by_year,
    x="年度_年",
    y=missing_ratio_by_year.columns.drop("年度_年"),
    title="年度別 欠損値割合の推移",
    markers=True,
)
fig.show()

,年度_年,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
0,2010,0.009102,0.009102,0.046682,0.056078,0.388139,0.405461,0.051674,0.010570
1,2011,0.012240,0.008540,0.059209,0.067179,0.395104,0.408767,0.062055,0.013664
2,2012,0.012538,0.008916,0.069936,0.079131,0.396768,0.407913,0.071329,0.013931
3,2013,0.013591,0.000815,0.068497,0.081816,0.403914,0.402011,0.068769,0.013862
4,2014,0.008466,0.000794,0.065608,0.080688,0.410847,0.395238,0.065873,0.008995
5,2015,0.012857,0.001286,0.070198,0.091026,0.418102,0.396760,0.070198,0.013371
6,2016,0.010003,0.002001,0.069767,0.098525,0.432858,0.395599,0.069767,0.010753
7,2017,0.011149,0.000242,0.073921,0.107610,0.441105,0.397479,0.073921,0.012603
8,2018,0.010407,0.000237,0.068827,0.073794,0.419820,0.373699,0.069300,0.011116
9,2019,0.008153,0.000466,0.062194,0.065921,0.400186,0.360820,0.062194,0.008852


'\nfor filename in files:\n    if filename not in missing_year_df:\n        all_df[filename] = pd.DataFrame()\n        missing_year_df[filename] = pd.DataFrame()\n    for year in years:\n        file_path = os.path.join(base_path, str(year), filename)\n        if os.path.exists(file_path):\n            df = pd.read_csv(file_path,header=1,na_values=["-","0"])\n            all_df[filename] = pd.concat([all_df[filename], df], ignore_index=True) # 全データ結合\n            # 各年の欠損率を計算\n            missing_rate = df.isnull().mean().to_frame(name="missing_rate")\n            missing_rate["year"] = year\n            missing_rate = missing_rate.reset_index().rename(columns={"index": "column_name"})\n            missing_year_df[filename] = pd.concat([missing_year_df[filename], missing_rate], ignore_index=True)\n#最新年に登録されている企業コードのみ抽出\nlatest_year = max(years)\nfile_path = os.path.join(base_path, str(latest_year), files[0])\ndf = pd.read_csv(file_path,header=1,na_values=["-","0"])\ncodes = df["コード"].to

In [ ]:
# 各カラムの欠損率推移を折れ線グラフで表示
for filename in files:
    fig = px.line(
        missing_year_df[filename],  # 最初のファイルのデータを使用
        x="year",
        y="missing_rate",
        color="column_name",   # 列ごとに色分け
        markers=True,
        title="各列の欠損率の推移"
    )
    fig.show()

### 年ごとの欠損率の傾向を確認（折れ線グラフ）

In [ ]:
# 年ごとの欠損率の傾向を確認（ヒートマップ）

for filename in missing_year_df:
    pivot_table = missing_year_df[filename].pivot(index="column_name", columns="year", values="missing_rate")
    fig = go.Figure(data=go.Heatmap(
        z=pivot_table.values,
        x=pivot_table.columns,
        y=pivot_table.index,
        colorscale="Viridis",
        colorbar=dict(title="Missing Rate")
    ))
    fig.update_layout(
        title=f"{filename} の欠損率ヒートマップ",
        xaxis_title="Year",
        yaxis_title="Column Name"
    )
    fig.show()

### 欠損率の詳細分析

In [ ]:
# コードごとに平均欠損率を計算する
missing_score = {}
for filename in files:
    # コードごとに欠損率を計算
    missing_score[filename] = pd.DataFrame()
    missing_score[filename] = (
        all_df[filename]
        .groupby("コード")
        .agg(lambda x: x.isnull().mean())  # 各列ごとの欠損率
        .mean(axis=1)                      # 列平均
        .reset_index(name="平均欠損率")
    )
    #pd.set_option('display.max_rows', None)
    #display(missing_score[filename].info())

In [ ]:
# 分析 fy-balance-sheet.csv
target_file = files[0]

fig = px.histogram(
    missing_score[target_file],  # 最初のファイルのデータを使用
    x="平均欠損率",
    nbins=50,
    title="全企業の欠損率分布"+files[0],
    labels={"平均欠損率": "平均欠損率"}
)
#fig.show()
# 特定範囲の欠損率を持つコードを抽出
threshold_low = 0.215
threshold_high= 0.225
filtered_codes = missing_score[target_file][
    (missing_score[target_file]["平均欠損率"] >= threshold_low) &
    (missing_score[target_file]["平均欠損率"] < threshold_high)
]
#display(filtered_codes)
# 列別欠損率
df_by_code = all_df[target_file].set_index("コード") # 企業コードを index にする
codes = filtered_codes["コード"].tolist() # 企業コードをリスト化
subset = df_by_code.loc[df_by_code.index.isin(codes)] # 企業コードのデータ抽出
#print(df_by_code)
col_missing = subset.isnull().mean().sort_values(ascending=False) # 列別欠損率
#display(col_missing)

#年次別の欠損率推移確認
df = all_df[target_file]
year_all = (
    df.groupby('年度')[['短期借入金', '長期借入金']].apply(lambda d: d.isnull().mean())
    .apply(lambda d: d)
    .reset_index()
)
# 欠損率の推移を可視化
fig = px.line(
    year_all,
    x="年度",
    y=["短期借入金", "長期借入金"],  # 複数列を同時に表示
    markers=True,
    title="各列の欠損率の推移"
)
#fig.show()
# 各企業の年次ごとの欠損率を計算し、"常時欠損" と "変化あり" を分類
status = {}
code_groups = subset.groupby("コード")
for code, g in code_groups:
    rate_by_year = g[['短期借入金','長期借入金']].isnull().mean(axis=1)  # 年行ごとの欠損率
    if rate_by_year.nunique() == 1 and rate_by_year.iloc[0] == 1.0:
        status[code] = 'always_missing'
    else:
        status[code] = 'variable'
pd.Series(status).value_counts()
# variable の一部を表示して具体的な年次パターンを確認
#for code, st in list(status.items())[:100]:
    #if st == 'variable':
        #print(code)
        #display(code_groups.get_group(code)[['年度','短期借入金','長期借入金']].head(20))
# 業種に偏りがあるか確認
codes = filtered_codes["コード"].tolist()
p = os.path.join(base_path, files_reference[0])
df = pd.read_csv(p,header=0,na_values=[])
# 全体データ
df_all = df.copy()
df_all["区分"] = "全体"
# subsetデータ
df_sub = df[df["コード"].isin(codes)].copy()
df_sub["区分"] = "抽出subset"
# 結合
df_combined = pd.concat([df_all, df_sub])
# Plotly Expressで可視化
fig = px.histogram(
    df_combined,
    x="33業種区分",
    color="区分",
    barmode="group",
    histnorm="percent",  # ← 割合表示
    title="33業種区分別の構成比比較（全体 vs 抽出subset）"
)
#fig.show()


In [ ]:
# 時価総額に偏りがあるか確認
# 時価総額の推定モデル：推定時価総額 = (PER * EPS) * (自己資本 / BPS) * (ROE / 10 + 営業CFマージン)
# 4つのファイルを結合
df_combined = pd.DataFrame()
for filename in files:
    df=all_df[filename]
    df_combined = pd.concat([df_combined, df], axis=0)
#display(df_combined)
df = df_combined[["コード","年度","純利益","EPS","株主資本","BPS","ROE","営業CFマージン"]].copy()
def combine_nonnull(series):
    #そのグループ内で最初に非NaNの値を返す
    return series.dropna().iloc[0] if series.notna().any() else np.nan

# 「コード」「年度」でグループ化して統合
df_merged = (
    df.groupby(["コード", "年度"], as_index=False)
      .agg(combine_nonnull)
      .sort_values(["コード", "年度"])
      .reset_index(drop=True)
)
#display(df_merged.head(1000))

In [ ]:
df_merged["推定時価総額"] = (
    (df_merged["純利益"] * df_merged["EPS"]) *
    (df_merged["株主資本"] / df_merged["BPS"]) *
    (df_merged["ROE"] / 10 + df_merged["営業CFマージン"])
)
#display(df_merged.head(1000))
df_all = df_merged.copy()
df_all["区分"] = "全体"
# subsetデータ
codes = filtered_codes["コード"].tolist()
df_sub = df_merged[df_merged["コード"].isin(codes)].copy()
df_sub["区分"] = "抽出subset"
# 結合
df_combined = pd.concat([df_all, df_sub])
# Plotly Expressで可視化
fig = px.histogram(
    df_combined,
    x="推定時価総額",
    color="区分",
    barmode="group",
    title="推定時価総額別の構成比比較（全体 vs 抽出subset）"
)
fig.show()

# OLD --- IGNORE ---

In [ ]:
# IR bankのデータから必要なデータを吸い上げる
import pandas as pd
import myLibStandard as myLib
myLib.set_DisplayOption()

# BSから必要なデータを取得
dfBalanceSheet = pd.DataFrame()
dl = ["fy-balance-sheet", "fy-cash-flow-statement", "fy-profit-and-loss", "fy-stock-dividend"]
for folder in range(2010, 2026):
    df = pd.read_csv(myLib.pathDataDomesticData + str(folder) + "/fy-balance-sheet.csv",\
    header=1, index_col=["コード", "年度"], dtype={"コード":"str"})
    dfBalanceSheet = pd.concat([dfBalanceSheet, df], axis=0, join="outer")
dfBalanceSheet = dfBalanceSheet.sort_index()
dfBalanceSheet = dfBalanceSheet.loc[~dfBalanceSheet.index.duplicated("last")]

# PLから必要なデータを取得
dfProftLoss = pd.DataFrame()
for folder in range(2010, 2026):
    df = pd.read_csv(myLib.pathDataDomesticData + str(folder) + "/" + "fy-profit-and-loss.csv",\
    header=1, index_col=["コード", "年度"], dtype={"コード":"str"})
    dfProftLoss = pd.concat([dfProftLoss, df], axis=0, join="outer")
dfProftLoss = dfProftLoss.sort_index()
dfProftLoss = dfProftLoss.loc[~dfProftLoss.index.duplicated("last")]

# SDから必要なデータを取得
dfStockDividend = pd.DataFrame()
for folder in range(2010, 2026):
    df = pd.read_csv(myLib.pathDataDomesticData + str(folder) + "/" + "fy-stock-dividend.csv",\
    header=1, index_col=["コード", "年度"], dtype={"コード":"str"})
    dfStockDividend = pd.concat([dfStockDividend, df], axis=0, join="outer")
dfStockDividend = dfStockDividend.loc[~dfStockDividend.index.duplicated("last")]
dfStockDividend = dfStockDividend.sort_index(level=["コード"])
#display(dfStockDividend)

dfAll = pd.concat([dfBalanceSheet, dfProftLoss], axis=1, join="outer")
dfAll = pd.concat([dfAll, dfStockDividend], axis=1, join="outer")
dfAll = dfAll.sort_index()
#display(dfAll.head(1000))
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv")


In [ ]:
# データがないセルを"-"からnullに変換する
import pandas as pd
import math
import myLibStandard as myLib
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv",index_col=["コード", "年度"], dtype="str")
dfAll = dfAll.sort_index()
for column in dfAll:
    dfAll[column] = dfAll[column].replace("-", math.nan)
    dfAll[column] = dfAll[column].replace(0, math.nan)

dfAll.to_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv")


In [ ]:
# データが0の場合はNANにする（ゼロ割り算エラーを避けるため）
import pandas as pd
import math
import myLibStandard as myLib
myLib.set_DisplayOption()
dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv", dtype={"コード":"str"})

for i in dfAll.index:
    if dfAll.loc[i, "EPS"] == 0:
        dfAll.loc[i, "EPS"] = math.nan
    if dfAll.loc[i, "BPS"] == 0:
        dfAll.loc[i, "BPS"] = math.nan
dfAll = dfAll.set_index(["コード", "年度"])
#display(dfAll.head(1000))
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv")

##### 銘柄名や業種を取得

In [ ]:
# シンボルリスト取得、保存
from stocksymbol import StockSymbol
api_key = '63ed951b-b643-48b1-bfe7-90c7a4f96349'
ss = StockSymbol(api_key)
symbol_list_japan = ss.get_symbol_list(market="Japan") # "us" or "america" will also work
df = symbol_list_japan
df = pd.DataFrame(df)
df.to_csv(myLib.pathDataSummaryDomesticData + "symbol.csv")

In [ ]:
# Code一覧から銘柄の情報を取得し、登録する。ないものはデータから削除する
import pandas as pd
import myLibStandard as myLib
import warnings
#warnings.simplefilter('ignore')
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv",index_col=["コード", "年度"], dtype={"コード":str})
dfCode = pd.read_csv(myLib.pathDataSummaryDomesticData + "CodeData.csv",usecols=["コード", "銘柄名", "市場・商品区分", "33業種区分"],\
index_col=["コード"], dtype=str)
dfAll["銘柄"] = None
dfAll["市場"] = None
dfAll["業種"] = None
dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
dlDelCode = []
for code in dlCode:
    try:
        dfAll.loc[(code,), "銘柄"] = dfCode.loc[code, "銘柄名"]
        dfAll.loc[(code,), "市場"] = dfCode.loc[code, "市場・商品区分"]
        dfAll.loc[(code,), "業種"] = dfCode.loc[code, "33業種区分"]
    except KeyError:
        dlDelCode.append(code)

dfAll = dfAll.reset_index("年度")
for dl in dlDelCode:
    dfAll = dfAll.drop(dl,axis=0)
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv")

##### 銘柄のテーマ情報を取得し登録する

In [ ]:
# テーマ情報をCSVからとってきてテーマフラグを立てる
import pandas as pd
import myLibStandard as myLib
import warnings
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv",index_col="コード", dtype={"コード":str})

def get_ThemeList(filname):
    df = pd.read_csv(myLib.pathDataDomesticData + filname, index_col="コード", dtype={"コード":str})
    return df

dlname = ["DOE採用銘柄", "円高メリット", "減配なし", "親子上場", "累進配当"]
for name in dlname:
    dfTheme = get_ThemeList(name + ".csv")
    for code in dfTheme.index:
        if code in dfAll.index:
            dfAll.loc[code, name] = 1

dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv")


## 分析値の計算、追加

##### データの登録（PER, PBR, MIX係数, 簡易理論株価, 安全域, 時価総額, 配当利回り, 資産価値を含めたPERを登録する）

In [ ]:
# PER, PBR, 簡易理論株価, 安全域, 配当利回り, MIX係数, 時価総額, を登録する (※長時間)
import pandas as pd
import math
import datetime
import calendar
from yahooquery import Ticker
import myLibStandard as myLib
import warnings
warnings.simplefilter('ignore', FutureWarning)
myLib.set_DisplayOption()

#dfSymbol = pd.read_csv(myLib.pathDataSummaryDomesticData + "symbol.csv", dtype={"symbol":str})
dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv",index_col=["コード", "年度"], dtype={"コード":"str"})
#dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv",index_col=["コード", "年度"], dtype={"コード":str})

dfAll["PER"] = math.nan
dfAll["PBR"] = math.nan
dfAll["簡易理論株価"] = math.nan
dfAll["安全域"] = math.nan
dfAll["配当利回り"] = math.nan
dfAll["MIX係数"] = math.nan
dfAll["時価総額"] = math.nan
dfAll["株価"] = math.nan
dfAll["営業利益率"] = math.nan
dfAll["経常利益率"] = math.nan
dfAll["純利益率"] = math.nan
dfAll["配当将来期待"] = round(dfAll["ROE"] - dfAll["純資産配当率"] ,1)

def get_StockValue(symbol):
    yqData = Ticker(symbol+".T")
    stockValue = yqData.history(start="2010-03-01")
    stockValue = stockValue.reset_index(level=["date",'symbol'])
    stockValue["date"] = stockValue["date"].apply(lambda x: pd.to_datetime(x).tz_localize(None))
    stockValue["date"] = pd.to_datetime(stockValue["date"])
    stockValue = stockValue.set_index("date")
    stockValue = stockValue["close"].resample("D").ffill()
    return stockValue
    #return pd.Series()

dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
i=0
for code in dlCode:
    if (i>=0) and (i<5000):
        print(code)
        dsStock = get_StockValue(str(code))
        for date in dfAll.loc[(code,)].index:
            dtDate = datetime.datetime.strptime(date, '%Y/%m')
            dtDate = datetime.date(dtDate.year, dtDate.month, calendar.monthrange(dtDate.year, dtDate.month)[1])
            try:
                stockValue = dsStock.loc[dtDate.strftime("%Y-%m-%d")]
            except KeyError:
                stockValue = math.nan
            eps = float(dfAll.loc[(code, date), "EPS"])
            bps = float(dfAll.loc[(code, date), "BPS"])
            if eps < 0:
                dfAll.loc[(code, date), "PER"] = math.nan
            else:
                dfAll.loc[(code, date), "PER"] = round(stockValue / eps, 2)
            dfAll.loc[(code, date), "簡易理論株価"] = round(bps + eps * 10,2)
            dfAll.loc[(code, date), "PBR"] = round(stockValue / bps, 2)
            dfAll.loc[(code, date), "配当利回り"] = round(float(dfAll.loc[(code, date), "一株配当"] / stockValue *100),2)
            dfAll.loc[(code, date), "安全域"] = round(dfAll.loc[(code, date), "簡易理論株価"] / stockValue, 2)
            dfAll.loc[(code, date), "MIX係数"] = dfAll.loc[(code, date), "PER"] * dfAll.loc[(code, date), "PBR"]
            dfAll.loc[(code, date), "時価総額"] = dfAll.loc[(code, date), "PER"] * dfAll.loc[(code, date), "純利益"]
            dfAll.loc[(code, date), "株価"] = stockValue
            dfAll.loc[(code, date), "営業利益率"] = round(dfAll.loc[(code, date), "営業利益"] / dfAll.loc[(code, date), "売上高"] * 100 ,2)
            dfAll.loc[(code, date), "経常利益率"] = round(dfAll.loc[(code, date), "経常利益"] / dfAll.loc[(code, date), "売上高"] * 100 ,2)
            dfAll.loc[(code, date), "純利益率"] = round(dfAll.loc[(code, date), "純利益"] / dfAll.loc[(code, date), "売上高"] * 100 ,2)
    i+=1
    if i%10==0:
        print(code,i)
        dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")

In [ ]:
# 資産価値を含めたPERを登録する (※長時間)
import pandas as pd
import math
import datetime
import calendar
from yahooquery import Ticker
import yfinance as yf
import myLibStandard as myLib
import warnings
#warnings.simplefilter('ignore', FutureWarning)
myLib.set_DisplayOption()

#dfSymbol = pd.read_csv(myLib.pathDataSummaryDomesticData + "symbol.csv", dtype={"symbol":"str"})
dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv",index_col=["コード", "年度"], dtype={"コード":"str"})
dfAll["資産価値を含めたPER"] = math.nan

dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
#display(dlCode)
i=0
for code in dlCode:
    if (i>=0) & (i<=5000):
        print(code)
        try:
            balance_sheet = Ticker(code+".T").balance_sheet().reset_index().set_index("asOfDate")
        except AttributeError:
            balance_sheet = Ticker(code+".T").balance_sheet()
        #display(balance_sheet)
        if type(balance_sheet) != str:
            for date in balance_sheet.index:
                dtDate = datetime.date(date.year,date.month,1)
                strDate = datetime.date.strftime(dtDate, "%Y/%m")
                try:
                    CurrentAssets = balance_sheet.loc[date, "CurrentAssets"]
                except KeyError:
                    CurrentAssets = math.nan
                try:
                    NetDebt = balance_sheet.loc[date, "NetDebt"]
                except KeyError:
                    NetDebt = balance_sheet.loc[date, "TotalAssets"] - balance_sheet.loc[date, "StockholdersEquity"]
                try:
                    AvailableForSaleSecurities = balance_sheet.loc[date, "AvailableForSaleSecurities"]
                except KeyError:
                    AvailableForSaleSecurities = math.nan
                netCash = CurrentAssets + NetDebt * 0.7 -AvailableForSaleSecurities
                #print(netCash)
                #print(date)
                #display(dfAll.loc[(code, )].index)
                try:
                    dfAll.loc[(code, strDate), "資産価値を含めたPER"] = round(\
                    (dfAll.loc[(code, strDate), "時価総額"] - netCash) / dfAll.loc[(code, strDate), "純利益"], 2)
                except KeyError:
                    dfAll.loc[(code, strDate),:] = math.nan
                    dfAll.loc[(code, strDate), "資産価値を含めたPER"] = round(\
                    (dfAll.loc[(code, strDate), "時価総額"] - netCash) / dfAll.loc[(code, strDate), "純利益"], 2)
                #print(code, strDate, dfAll.loc[(code, strDate), "資産価値を含めたPER"])
    i+=1
    if i%10==0:
        print(i)
        dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")
        #break
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")

## 最新日付のファイルの準備

In [ ]:
# 最新日付のみのデータフレームを作成する（※中時間）
import pandas as pd
import myLibStandard as myLib
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv",parse_dates=[1],index_col=["コード", "年度"], dtype={"コード":str})
dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
dfMaxDate = pd.DataFrame()
for code in dlCode:
    maxDate = dfAll.loc[(code,)].index.max()
    #display(dfAll.loc[(code,maxDate)])
    #print(code)
    if maxDate.year >= 2024:
        ds = dfAll.loc[(code,maxDate)].to_frame().T
        dfMaxDate = pd.concat([dfMaxDate,ds],axis=0)
dfMaxDate.index.names = ["コード", "年度"]
dfMaxDate.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTableMaxDate.csv")